In [ ]:

import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC

plt.style.use("seaborn")


In [ ]:

# Load image and apply morphological transformations for feature enhancement
def load_transformed_image(path):
    image = cv.imread(path, 0)
    kernel = np.ones((5,5), np.uint8)
    img_erosion = cv.erode(image, kernel, iterations=1)
    img_dilation = cv.dilate(img_erosion, kernel, iterations=1)
    plt.imshow(cv.cvtColor(img_dilation, cv.COLOR_BGR2RGB))
    plt.show()
    return img_dilation

image_path = 'line1.jpg'
image = load_transformed_image(image_path)


In [ ]:

# Advanced segmentation using Otsu's thresholding
def otsu_segmentation(image):
    _, otsu_thresh = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    plt.imshow(otsu_thresh, cmap='gray')
    plt.show()
    return otsu_thresh

segmented_image = otsu_segmentation(image)


In [ ]:

# Prepare multilabel data and train MultiOutputClassifier
data = segmented_image.flatten()
labels = np.where(data > 0, 1, 0).reshape(-1, 1)
labels = np.hstack([labels, 1-labels])  # Create multilabel targets

x_train, x_test, y_train, y_test = train_test_split(data.reshape(-1, 1), labels, test_size=0.3, random_state=42)

# Train multioutput SVM classifier
svm = SVC(kernel='poly')
multilabel_classifier = MultiOutputClassifier(svm, n_jobs=-1)
multilabel_classifier.fit(x_train, y_train)
